In [ ]:
!pip install torch pennylane pennylane-qiskit


INFO: pip is looking at multiple versions of pennylane-qiskit to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.0/930.0 kB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.9/249.9 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.8/340.8 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.1 

In [ ]:
!pip install semantic_version


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pennylane as qml
from sklearn.metrics import f1_score, precision_score, recall_score
import numpy as np

# Define the quantum device
dev = qml.device('qiskit.aer', wires=4)  # Using 4 qubits

# Define a quantum circuit
@qml.qnode(dev)
def quantum_circuit(params):
    for i in range(4):
        qml.RX(params[i], wires=i)  # Rotations based on input parameters
    return [qml.expval(qml.PauliZ(i)) for i in range(4)]  # Return the expectation values

class HybridModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(HybridModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)  # Only one hidden layer
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size + 4, num_classes)  # Adjust output size for 27 classes

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)

        # Prepare parameters for the quantum circuit
        params = x[:, :4]  # Take first 4 features for quantum circuit

        # Execute quantum circuit for each element in the batch
        quantum_output = []
        for p in params.detach().cpu().numpy():  # Detach the tensor and move it to CPU for numpy conversion
            output = quantum_circuit(p)  # Get the quantum circuit output
            print(f"Quantum circuit output for parameters {p}: {output}")  # Debugging output
            quantum_output.append(output)  # Append the result

        # Flatten quantum_output to ensure it's a 2D array
        quantum_output = np.array(quantum_output)  # Convert to a NumPy array
        quantum_output = quantum_output.reshape(-1, 4)  # Ensure it's 2D, shape (batch_size, 4)

        # Convert quantum output to tensor and ensure it’s on the same device
        quantum_output = torch.tensor(quantum_output, dtype=torch.float32, device=x.device)

        # Concatenate classical and quantum outputs
        x = torch.cat((x, quantum_output), dim=1)  # Combine the outputs
        return self.fc2(x)  # Final classification layer for 27 classes

# Load your dataset
data = pd.read_csv('EDD_embedding.csv')  # Replace with your file path
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values
y = y - y.min()  # Ensure y starts from 0
num_classes = 27  # Set number of classes to 27

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocessing: Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train, X_test = torch.tensor(X_train, dtype=torch.float32), torch.tensor(X_test, dtype=torch.float32)
y_train, y_test = torch.tensor(y_train, dtype=torch.long), torch.tensor(y_test, dtype=torch.long)

# Create DataLoader for batch processing
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=32, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=32, shuffle=False)

# Instantiate the model, loss function, and optimizer
input_size = 1024  # Adjust according to your input features
hidden_size = 128  # Single hidden layer size
model = HybridModel(input_size, hidden_size, num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Specify the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Training loop
num_epochs = 17
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move to device

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

# Testing loop with metrics calculation
model.eval()
all_labels = []
all_predictions = []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move to device
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)

        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

# Calculate accuracy, precision, recall, and F1 score
accuracy = (torch.tensor(all_predictions) == torch.tensor(all_labels)).sum().item() / len(all_labels)
precision = precision_score(all_labels, all_predictions, average='weighted')
recall = recall_score(all_labels, all_predictions, average='weighted')
f1 = f1_score(all_labels, all_predictions, average='weighted')

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


/usr/local/lib/python3.10/dist-packages/pennylane/__init__.py:184: PennyLaneDeprecationWarning: QubitDevice will no longer be accessible top level. Please access the class as pennylane.devices.QubitDevice
  warn(


Streaming output truncated to the last 5000 lines.
Quantum circuit output for parameters [0.        2.3855104 5.925831  3.564193 ]: [array(1.), array(-0.74414062), array(0.94335938), array(-0.89257812)]
Quantum circuit output for parameters [0.        0.        0.3750569 2.4562116]: [array(1.), array(1.), array(0.94140625), array(-0.75976562)]
Quantum circuit output for parameters [0.        1.9856722 4.6177664 0.       ]: [array(1.), array(-0.36132812), array(-0.08789062), array(1.)]
Quantum circuit output for parameters [0.        3.4745378 1.3707926 0.       ]: [array(1.), array(-0.94335938), array(0.26171875), array(1.)]
Quantum circuit output for parameters [1.4808662 1.6428518 0.        4.0244446]: [array(0.12304688), array(-0.09375), array(1.), array(-0.62890625)]
Quantum circuit output for parameters [0. 0. 0. 0.]: [array(1.), array(1.), array(1.), array(1.)]
Quantum circuit output for parameters [1.6390666 1.8934753 0.        0.       ]: [array(-0.09375), array(-0.29882812), a